In [ ]:
import os
import sys
import time
import pickle

In [39]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from dotenv import load_dotenv


import torch
import torch.nn as nn

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MultiLabelBinarizer

from cuml.preprocessing import MinMaxScaler as CumlMinMaxScaler
from cuml.feature_extraction.text import TfidfVectorizer as CumlTfidfVectorizer
import cupy as cp
import cudf
from scipy.sparse import csr_matrix
from scipy.stats import entropy

from gensim.corpora.dictionary import Dictionary
from gensim.models.coherencemodel import CoherenceModel

In [ ]:
# Загрузка переменных окружения из файла .env
load_dotenv()

In [41]:
# Получение текущей директории
current_dir = os.getcwd()
# Получение корневой директории проекта
project_root = os.path.dirname(os.path.dirname(current_dir))

In [42]:
# Добавление корневой директории проекта в sys.path для импорта модулей
if project_root not in sys.path:
    sys.path.append(project_root)

In [43]:
# Пути к файлам с исходными данными
df_raw_json_path = os.path.join(project_root, 'data', 'raw', 'steam_games_data.json')
df_raw_csv_path = os.path.join(project_root, 'data', 'raw', 'steam_games_data.csv')

In [44]:
# Пути к файлам с обработанными данными
df_processed_json_path = os.path.join(project_root, 'data', 'processed', 'steam_games_data.json')
df_processed_csv_path = os.path.join(project_root, 'data', 'processed', 'steam_games_data.csv')

In [45]:
# Пути к результатам param grid
file_lda = "manual_coherence_results_lda.csv"
file_nmf = "manual_coherence_results_nmf.csv"

In [46]:
# Путь к итоговой модели
model_path = "best_model_manual_coherence.pkl"

In [ ]:
# Определение устройства Torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"⚙️ Используемое устройство: {device}")

In [ ]:
# Загрузка обработанных данных
print("🔄 Загрузка данных...")
df = pd.read_json(df_processed_json_path)
print("✅ Данные загружены.")

In [49]:
def reduce_dataset(df, percentage=0.1):
    """Уменьшает размер датасета до указанного процента.

    Аргументы:
        df (pd.DataFrame): Исходный датасет.
        percentage (float): Процент размера датасета, который нужно оставить (от 0 до 1).

    Возвращает:
        pd.DataFrame: Уменьшенный датасет.

    Вызывает ValueError, если percentage не находится в диапазоне [0, 1].
    """
    if not 0 <= percentage <= 1:
        raise ValueError("❌ Процент должен быть в диапазоне от 0 до 1")

    print(f"📉 Уменьшение датасета до {percentage * 100}%...")
    df_sorted = df.sort_values(by='estimated_owners', ascending=False)
    num_rows = int(len(df_sorted) * percentage)
    reduced_df = df_sorted.head(num_rows)
    print(f"✅ Датасет уменьшен до {len(reduced_df)} строк.")
    return reduced_df

In [50]:
#df = reduce_dataset(df, percentage=0.5)

In [ ]:
df.shape

In [ ]:
# Разделение на обучающую и тестовую выборки
print("➗ Разделение данных на обучающую и тестовую выборки...")
train_df, test_df = train_test_split(df, test_size=0.001, random_state=42)
print("✅ Данные разделены.")

In [ ]:
test_df.shape

In [54]:
class TorchLDA(nn.Module):
    """Реализация LDA на PyTorch.

    Инициализирует и обучает модель LDA, используя EM-алгоритм.

    Аргументы:
        n_topics (int): Количество тем.
        n_vocab (int): Размер словаря.
        device (torch.device): Устройство для вычислений (CPU или GPU).
        alpha (float): Параметр регуляризации для распределения документов по темам.
        beta (float): Параметр регуляризации для распределения тем по словам.
        max_iterations (int): Максимальное количество итераций EM-алгоритма.
        tolerance (float): Порог сходимости для EM-алгоритма.
    """
    def __init__(self, n_topics, n_vocab, device, alpha=0.1, beta=0.01, max_iterations=100, tolerance=1e-4):
        super().__init__()
        self.n_topics = n_topics
        self.n_vocab = n_vocab
        self.device = device
        self.alpha = alpha
        self.beta = beta
        self.max_iterations = max_iterations
        self.tolerance = tolerance
        self.topic_term_matrix = nn.Parameter(torch.randn(n_topics, n_vocab, device=device).abs())
        self.doc_topic_matrix = None
        self.norm_topic_term_matrix = None

    def initialize_parameters(self, docs):
        """Инициализирует параметры модели LDA.

        Инициализирует матрицу распределения документов по темам и матрицу распределения тем по словам случайными значениями.

        Аргументы:
            docs (torch.Tensor): Матрица документов (размерность: количество документов x размер словаря).
        """
        self.doc_topic_matrix = torch.rand(docs.shape[0], self.n_topics, device=self.device).abs()
        self.topic_term_matrix.data = torch.randn(self.n_topics, self.n_vocab, device=self.device).abs()

    def fit(self, docs, log=False):
        """Обучает модель LDA на основе предоставленных документов, используя EM-алгоритм.

        Аргументы:
            docs (torch.Tensor): Матрица документов (размерность: количество документов x размер словаря).
            log (bool, optional): Включает вывод логов во время обучения. По умолчанию False.

        Возвращает:
            TorchLDA: Обученная модель LDA.
        """
        if log: print("LDA Fit started")
        self.initialize_parameters(docs)
        docs = docs.to(self.device)
        prev_likelihood = float('-inf')
        for iteration in range(self.max_iterations):
            doc_topic_distribution = self.expect(docs)
            self.topic_term_matrix = self.maximize(docs, doc_topic_distribution)
            current_likelihood = self.likelihood(docs, doc_topic_distribution)
            if log: print(f"Iteration {iteration+1}, Likelihood {current_likelihood:.2f}")
            if abs(current_likelihood - prev_likelihood) < self.tolerance:
                if log: print("LDA Converged")
                break
            prev_likelihood = current_likelihood
        self.norm_topic_term_matrix = self.normalize(self.topic_term_matrix)
        if log: print("LDA Fit ended")
        return self

    def expect(self, docs):
        """Выполняет E-шаг EM-алгоритма для LDA.

        Оценивает распределение документов по темам, учитывая текущую матрицу распределения тем по словам.

        Аргументы:
            docs (torch.Tensor): Матрица документов.

        Возвращает:
            torch.Tensor: Матрица распределения документов по темам.
        """
        doc_topic_distribution = torch.matmul(docs, self.topic_term_matrix.T) + self.alpha
        doc_topic_distribution = self.normalize(doc_topic_distribution)
        return doc_topic_distribution

    def maximize(self, docs, doc_topic_distribution):
        """Выполняет M-шаг EM-алгоритма для LDA.

        Обновляет матрицу распределения тем по словам, основываясь на оцененном распределении документов по темам.

        Аргументы:
            docs (torch.Tensor): Матрица документов.
            doc_topic_distribution (torch.Tensor): Матрица распределения документов по темам.

        Возвращает:
            torch.Tensor: Обновленная матрица распределения тем по словам.
        """
        topic_term_matrix = torch.matmul(doc_topic_distribution.T, docs) + self.beta
        return topic_term_matrix

    def likelihood(self, docs, doc_topic_distribution):
        """Вычисляет логарифмическое правдоподобие для текущей итерации EM-алгоритма.

        Используется для оценки сходимости алгоритма.

        Аргументы:
            docs (torch.Tensor): Матрица документов.
            doc_topic_distribution (torch.Tensor): Матрица распределения документов по темам.

        Возвращает:
            float: Значение логарифмического правдоподобия.
        """
        log_likelihood = torch.sum(docs * torch.log(torch.matmul(doc_topic_distribution, self.normalize(self.topic_term_matrix))))
        return log_likelihood.item()

    def normalize(self, matrix):
        """Нормализует матрицу, приводя суммы строк к единице.

        Аргументы:
            matrix (torch.Tensor): Матрица для нормализации.

        Возвращает:
            torch.Tensor: Нормализованная матрица.
        """
        row_sums = matrix.sum(axis=1, keepdim=True)
        return matrix / row_sums

    def transform(self, docs):
        """Преобразует новые документы в векторное представление в пространстве тем.

        Использует обученную модель LDA для проецирования документов в тематическое пространство.

        Аргументы:
            docs (torch.Tensor): Матрица новых документов.

        Возвращает:
            torch.Tensor: Матрица распределения документов по темам для новых документов.

        Вызывает ValueError, если модель LDA еще не обучена.
        """
        if self.norm_topic_term_matrix is None:
            raise ValueError("❌ Модель LDA еще не обучена.")
        docs = docs.to(self.device)
        doc_topic_distribution = torch.matmul(docs, self.norm_topic_term_matrix.T) + self.alpha
        return self.normalize(doc_topic_distribution)

In [55]:
def vectorize_owners(df, method='log_scale', scaler=None):
    """Векторизует данные о владельцах игр.

    Применяет различные методы векторизации к данным о предполагаемом количестве владельцев игр.

    Аргументы:
        df (pd.DataFrame): DataFrame, содержащий столбец 'estimated_owners'.
        method (str, optional): Метод векторизации: 'log_scale' или 'standard'. По умолчанию 'log_scale'.
        scaler (CumlMinMaxScaler, optional): Обученный scaler для масштабирования данных. Если None, scaler не применяется.

    Возвращает:
        np.ndarray: Векторизованные данные о владельцах.

    Вызывает ValueError, если указан недопустимый метод векторизации.
    """
    owners = df['estimated_owners'].values.reshape(-1, 1)
    owners = np.array(owners, dtype=float)
    owners = np.nan_to_num(owners, nan=0)
    if method == 'log_scale':
        owners = np.log1p(owners)
        if scaler is not None:
           owners = scaler.transform(owners)
        owners_weighted = owners * (1 + (owners * 2))
        return owners_weighted
    elif method == 'standard':
        if scaler is not None:
           owners = scaler.transform(owners)
        return owners
    else:
        raise ValueError("❌ Недопустимый метод векторизации владельцев.")

In [56]:
def vectorize_tags(df, multilabel_params=None):
    """Векторизует теги игр, используя MultiLabelBinarizer.

    Преобразует список тегов для каждой игры в многомерный бинарный вектор.

    Аргументы:
        df (pd.DataFrame): DataFrame, содержащий столбец 'all_tags' со списками тегов.
        multilabel_params (dict, optional): Параметры для MultiLabelBinarizer. По умолчанию {'sparse_output': False}.

    Возвращает:
        tuple: Кортеж, содержащий:
            - np.ndarray: Векторизованные теги.
            - MultiLabelBinarizer: Обученный объект MultiLabelBinarizer.
    """
    default_params = {'sparse_output': False}
    params = multilabel_params if multilabel_params else default_params
    mlb = MultiLabelBinarizer(**params)
    mlb.fit(df['all_tags'])
    tags_vectorized = mlb.transform(df['all_tags'])
    return tags_vectorized, mlb

In [57]:
def vectorize_descriptions(df, nmf_params=None, lda_params=None, vectorizer_cuml=None):
    """Векторизует описания игр, используя TF-IDF и NMF или LDA.

    Использует CumlTfidfVectorizer для преобразования текстовых описаний в векторы TF-IDF,
    а затем применяет NMF или LDA для тематического моделирования.

    Аргументы:
        df (pd.DataFrame): DataFrame, содержащий столбец 'short_description_clean' с очищенными описаниями.
        nmf_params (dict, optional): Параметры для NMF. Если указаны, используется NMF.
        lda_params (dict, optional): Параметры для LDA. Если указаны, используется LDA.
        vectorizer_cuml (CumlTfidfVectorizer, optional): Обученный CumlTfidfVectorizer.

    Возвращает:
        tuple: Кортеж, содержащий:
            - np.ndarray: Векторизованные описания (тематические векторы).
            - NMF или LatentDirichletAllocation: Обученная модель NMF или LDA.

    Вызывает ValueError, если не предоставлен обученный CumlTfidfVectorizer или не указаны параметры nmf_params или lda_params.
    """
    if vectorizer_cuml is None:
        raise ValueError("❌ Необходимо предоставить обученный CumlTfidfVectorizer.")
    desc_vectorized_cuml = vectorizer_cuml.transform(df['short_description_clean'])
    desc_vectorized_cuml_cpu = desc_vectorized_cuml.get()
    data = cp.asnumpy(desc_vectorized_cuml_cpu.data)
    indices = cp.asnumpy(desc_vectorized_cuml_cpu.indices)
    indptr = cp.asnumpy(desc_vectorized_cuml_cpu.indptr)
    shape = desc_vectorized_cuml_cpu.shape
    desc_vectorized_cpu = csr_matrix((data, indices, indptr), shape=shape)

    if nmf_params:
        nmf = NMF(**nmf_params)
        nmf_vectorized = nmf.fit_transform(desc_vectorized_cpu)
        return nmf_vectorized, nmf
    elif lda_params:
        lda = LatentDirichletAllocation(**lda_params)
        lda_vectorized = lda.fit_transform(desc_vectorized_cpu)
        return lda_vectorized, lda
    else:
        raise ValueError("❌ Необходимо указать nmf_params или lda_params")

In [58]:
def calculate_topic_coherence(model, vectorizer, texts):
    """Вычисляет когерентность темы модели.

    Использует UMass coherence для оценки качества тематической модели.

    Аргументы:
        model: Обученная тематическая модель (NMF или LDA) с атрибутом 'components_'.
        vectorizer: Обученный TF-IDF векторизатор с методом 'get_feature_names'.
        texts (list): Список текстов, использованных для обучения модели.

    Возвращает:
        float: Значение когерентности темы. Возвращает -999 в случае ошибки.
    """
    try:
        feature_names_cuml = vectorizer.get_feature_names()

        if isinstance(feature_names_cuml, cudf.core.series.Series):
            feature_names = feature_names_cuml.to_pandas().tolist()
        elif not isinstance(feature_names_cuml, list):
            return -999

        if hasattr(model, 'components_') and feature_names is not None:
            topic_vectors = model.components_
            if isinstance(topic_vectors, cp.ndarray):
                topic_vectors_np = topic_vectors.get()
            else:
                topic_vectors_np = topic_vectors

            top_words_idx = topic_vectors_np.argsort()[:, ::-1]
            top_words = [[feature_names[i] for i in topic_word_idx[:10]] for topic_word_idx in top_words_idx]

            dictionary = Dictionary([text.split() for text in texts])
            tokenized_texts = [text.split() for text in texts]

            cm = CoherenceModel(topics=top_words, texts=tokenized_texts, dictionary=dictionary, coherence='u_mass')

            coherence_score = cm.get_coherence()
            return coherence_score
        else:
            return -999
    except Exception:
        return -999

In [59]:
def calculate_topic_diversity(model):
    """Вычисляет разнообразие тем на основе косинусного расстояния.

    Измеряет разнообразие тем в модели, рассчитывая среднее косинусное расстояние между векторами тем.

    Аргументы:
        model: Обученная тематическая модель (NMF или LDA) с атрибутом 'components_'.

    Возвращает:
        float: Среднее косинусное расстояние между темами, представляющее разнообразие тем.
               Возвращает -1, если модель не имеет атрибута 'components_' или количество тем меньше 2.
    """
    if not hasattr(model, 'components_'):
        print("⚠️ Модель не имеет атрибута components_.")
        return -1

    topic_vectors = model.components_
    if isinstance(topic_vectors, cp.ndarray):
        topic_vectors_np = topic_vectors.get()
    else:
        topic_vectors_np = topic_vectors

    if topic_vectors_np.shape[0] < 2:
        print("⚠️ Менее двух тем. Невозможно вычислить разнообразие.")
        return -1

    num_topics = topic_vectors_np.shape[0]
    total_similarity = 0
    num_pairs = 0

    for i in range(num_topics):
      for j in range(i+1, num_topics):
         similarity = cosine_similarity(topic_vectors_np[i].reshape(1, -1), topic_vectors_np[j].reshape(1, -1))[0][0]
         total_similarity += similarity
         num_pairs += 1

    if num_pairs == 0:
      return -1

    average_similarity = total_similarity / num_pairs
    average_distance = 1 - average_similarity
    return average_distance

In [60]:
def calculate_intra_topic_diversity(model, feature_names, num_top_words=10):
    """Вычисляет разнообразие слов внутри каждой темы, используя энтропию.

    Оценивает, насколько разнообразны слова внутри каждой темы, используя энтропию распределения вероятностей топ-слов.

    Аргументы:
        model: Обученная тематическая модель (NMF или LDA) с атрибутом 'components_'.
        feature_names (list): Список названий признаков (слов) из TF-IDF векторизатора.
        num_top_words (int, optional): Количество топ-слов, учитываемых для расчета энтропии. По умолчанию 10.

    Возвращает:
        float: Средняя энтропия по всем темам, представляющая внутритопиковое разнообразие.
               Возвращает -1, если модель не имеет атрибута 'components_' или нет тем для расчета.
    """
    if not hasattr(model, 'components_'):
        print("⚠️ Модель не имеет атрибута components_.")
        return -1

    topic_vectors = model.components_
    if isinstance(topic_vectors, cp.ndarray):
        topic_vectors_np = topic_vectors.get()
    else:
        topic_vectors_np = topic_vectors
    num_topics = topic_vectors_np.shape[0]

    if num_topics == 0:
        print("⚠️ Нет тем для расчета разнообразия.")
        return -1

    topic_entropies = []
    for topic in topic_vectors_np:
        top_word_indices = np.argsort(topic)[::-1][:num_top_words]
        top_word_probabilities = topic[top_word_indices]
        normalized_probabilities = top_word_probabilities / np.sum(top_word_probabilities)
        topic_entropy = entropy(normalized_probabilities, base=2)
        topic_entropies.append(topic_entropy)

    return np.mean(topic_entropies) if topic_entropies else -1

In [61]:
def display_topics(model, feature_names, num_top_words=10):
    """Выводит наиболее значимые слова для каждой темы.

    Отображает топ-слова, которые наиболее важны для каждой темы в тематической модели.

    Аргументы:
        model: Обученная тематическая модель (NMF или LDA) с атрибутом 'components_'.
        feature_names (list): Список названий признаков (слов) из TF-IDF векторизатора.
        num_top_words (int, optional): Количество топ-слов для отображения для каждой темы. По умолчанию 10.
    """
    for topic_idx, topic in enumerate(model.components_):
        print(f"   Тема #{topic_idx}:", end=' ')
        top_word_indices = topic.argsort()[::-1][:num_top_words]
        top_words = [feature_names[i] for i in top_word_indices]
        print(" ".join(top_words))
    print()

In [62]:
def display_topics_with_diversity(model, feature_names, num_top_words=25, num_display_words = 10):
    """Выводит топ-слова для каждой темы и их энтропию.

    Отображает наиболее значимые слова для каждой темы, а также значение энтропии для оценки разнообразия слов в теме.

    Аргументы:
        model: Обученная тематическая модель (NMF или LDA) с атрибутом 'components_'.
        feature_names (list): Список названий признаков (слов) из TF-IDF векторизатора.
        num_top_words (int, optional): Количество топ-слов, рассматриваемых для расчета энтропии и отбора. По умолчанию 25.
        num_display_words (int, optional): Количество топ-слов для отображения для каждой темы. По умолчанию 10.
    """
    if not hasattr(model, 'components_'):
        print("⚠️ Модель не имеет атрибута components_.")
        return

    topic_vectors = model.components_
    if isinstance(topic_vectors, cp.ndarray):
        topic_vectors_np = topic_vectors.get()
    else:
        topic_vectors_np = topic_vectors

    for topic_idx, topic in enumerate(topic_vectors_np):
        print(f"   Тема #{topic_idx}. ", end=' ')
        top_word_indices = np.argsort(topic)[::-1][:num_top_words]
        top_words = [feature_names[i] for i in top_word_indices]
        print(f"   Топ-{num_display_words} слов: {' '.join(top_words[:num_display_words])}")
        normalized_probabilities = topic[top_word_indices] / np.sum(topic[top_word_indices])
        topic_entropy = entropy(normalized_probabilities, base=2)
        print(f"   Энтропия темы: {topic_entropy:.4f}")
    print()

In [63]:
class CombinedVectorizer(BaseEstimator, TransformerMixin):
    """Комбинированный векторизатор для обработки различных типов признаков.

    Векторизует данные о владельцах, теги и текстовые описания игр, используя различные методы и векторизаторы.

    Аргументы:
        owners_method (str, optional): Метод векторизации для данных о владельцах ('log_scale' или 'standard'). По умолчанию 'log_scale'.
        multilabel_params (dict, optional): Параметры для MultiLabelBinarizer. По умолчанию None.
        nmf_params (dict, optional): Параметры для NMF. Если указаны, используется NMF для векторизации описаний. По умолчанию None.
        lda_params (dict, optional): Параметры для LDA. Если указаны, используется LDA для векторизации описаний. По умолчанию None.
        tag_weight (float, optional): Вес, применяемый к векторизованным тегам. По умолчанию 1.0.
        tfidf_cuml_params (dict, optional): Параметры для CumlTfidfVectorizer. По умолчанию None.
    """
    def __init__(self, owners_method='log_scale', multilabel_params=None, nmf_params=None, lda_params=None, tag_weight=1.0, tfidf_cuml_params=None):
        self.owners_method = owners_method
        self.multilabel_params = multilabel_params
        self.nmf_params = nmf_params
        self.lda_params = lda_params
        self.tag_weight = tag_weight
        self.tfidf_cuml_params = tfidf_cuml_params if tfidf_cuml_params else {}
        self.tfidf_cuml = CumlTfidfVectorizer(**self.tfidf_cuml_params)
        self.mlb = None
        self.nmf = None
        self.lda = None
        self.tfidf_feature_names_out_ = None
        self.scaler = CumlMinMaxScaler()
        self.transformed_owners_vectors = None
        self.transformed_tags_vectors = None
        self.transformed_desc_vectors = None
        self.transformed_combined_vectors = None

    def fit(self, X, y=None):
        """Обучает векторизатор на предоставленных данных.

        Выполняет векторизацию владельцев, тегов и описаний, а также обучает внутренние векторизаторы и скалеры.

        Аргументы:
            X (pd.DataFrame): DataFrame, содержащий данные для векторизации ('estimated_owners', 'all_tags', 'short_description_clean').
            y (None): Не используется, нужен для совместимости API scikit-learn.

        Возвращает:
            CombinedVectorizer: Обученный векторизатор.
        """
        self.owners_vectors = vectorize_owners(X, method=self.owners_method)
        self.tags_vectors, self.mlb = vectorize_tags(X, multilabel_params=self.multilabel_params)
        if isinstance(self.tags_vectors, cp.sparse.csr_matrix):
            print("ℹ️ Векторы тегов - cupy sparse matrix, преобразование в numpy...")
            self.tags_vectors = np.array(cp.asnumpy(self.tags_vectors.todense()), dtype=np.float64)
        if self.tags_vectors.ndim == 1:
            self.tags_vectors = self.tags_vectors.reshape(-1, 1)

        cleaned_descriptions = X['short_description_clean'].str.lower()
        self.tfidf_cuml.fit(cleaned_descriptions)
        self.tfidf_feature_names_out_ = [word for word, index in sorted(self.tfidf_cuml.vocabulary_.to_pandas().items(), key=lambda item: item[1])]

        if self.nmf_params and self.lda_params is None:
            self.desc_vectors, self.nmf = vectorize_descriptions(X, nmf_params=self.nmf_params, vectorizer_cuml=self.tfidf_cuml)
            self.lda = None
        elif self.lda_params and self.nmf_params is None:
             self.desc_vectors, self.lda = vectorize_descriptions(X, lda_params=self.lda_params, vectorizer_cuml=self.tfidf_cuml)
             self.nmf = None
        else:
            raise ValueError("❌ Необходимо указать nmf_params или lda_params")
        print("✅ Векторизация описаний завершена")

        if self.nmf and hasattr(self.nmf, 'components_'):
             if np.isnan(self.nmf.components_).any():
                print("⚠️ Обнаружены NaN значения в self.nmf.components_ в fit()!")
        if self.lda and hasattr(self.lda, 'components_'):
             if np.isnan(self.lda.components_).any():
                 print("⚠️ Обнаружены NaN значения в self.lda.components_ в fit()!")

        owners_vectors = vectorize_owners(X, method=self.owners_method)
        self.scaler.fit(owners_vectors)
        return self

    def transform(self, X, y=None):
        """Трансформирует входные данные в комбинированные векторы признаков.

        Использует обученные векторизаторы и скалеры для преобразования данных о владельцах, тегов и описаний в единое векторное представление.

        Аргументы:
            X (pd.DataFrame): DataFrame, содержащий данные для трансформации.
            y (None): Не используется, нужен для совместимости API scikit-learn.

        Возвращает:
            np.ndarray: Матрица комбинированных векторов признаков.
        """
        owners_vectors = vectorize_owners(X, method=self.owners_method, scaler=self.scaler)
        owners_vectors = owners_vectors.reshape(owners_vectors.shape[0], -1)
        tags_vectors = self.mlb.transform(X['all_tags'])

        tag_weight = self.tag_weight
        tags_vectors_weighted = tags_vectors * tag_weight
        tags_vectors = tags_vectors_weighted

        tfidf_transformed_cuml = self.tfidf_cuml.transform(X['short_description_clean'])
        tfidf_transformed_cpu = tfidf_transformed_cuml.get()
        data = cp.asnumpy(tfidf_transformed_cpu.data)
        indices = cp.asnumpy(tfidf_transformed_cpu.indices)
        indptr = cp.asnumpy(tfidf_transformed_cpu.indptr)
        shape = tfidf_transformed_cpu.shape
        tfidf_transformed = csr_matrix((data, indices, indptr), shape=shape)

        desc_vectors = None
        if self.nmf_params:
            desc_vectors = self.nmf.transform(tfidf_transformed)
        elif self.lda_params:
            desc_vectors = self.lda.transform(tfidf_transformed)

        if desc_vectors is not None and desc_vectors.shape[0] != owners_vectors.shape[0]:
            raise ValueError(f"❌ Несовпадение количества образцов между векторами владельцев и описаний: {owners_vectors.shape[0]} vs {desc_vectors.shape[0]}")

        self.transformed_owners_vectors = owners_vectors
        self.transformed_tags_vectors = tags_vectors
        self.transformed_desc_vectors = desc_vectors

        combined_vectors = np.hstack([owners_vectors, tags_vectors.toarray() if hasattr(tags_vectors, 'toarray') else tags_vectors, desc_vectors])
        self.transformed_combined_vectors = combined_vectors

        return combined_vectors

    def get_params(self, deep=True):
        """Возвращает параметры векторизатора.

        Возвращает словарь параметров данного векторизатора, включая параметры для всех внутренних векторизаторов и методов.

        Аргументы:
            deep (bool, optional): Если True, также возвращает параметры для вложенных объектов, которые являются оценщиками. По умолчанию True.

        Возвращает:
            dict: Словарь параметров векторизатора.
        """
        return {
            'owners_method': self.owners_method,
            'multilabel_params': self.multilabel_params,
             'nmf_params': self.nmf_params,
            'lda_params': self.lda_params,
            'tag_weight': self.tag_weight,
            'tfidf_cuml_params': self.tfidf_cuml_params
        }

    def set_params(self, **params):
        """Устанавливает параметры векторизатора.

        Позволяет установить параметры векторизатора после инициализации.

        Аргументы:
            **params: Параметры векторизатора в виде keyword arguments.

        Возвращает:
            CombinedVectorizer: Векторизатор с установленными параметрами.
        """
        if 'owners_method' in params:
            self.owners_method = params['owners_method']
        if 'multilabel_params' in params:
            self.multilabel_params = params['multilabel_params']
        if 'nmf_params' in params:
            self.nmf_params = params['nmf_params']
        if 'lda_params' in params:
             self.lda_params = params['lda_params']
        if 'tag_weight' in params:
            self.tag_weight = params['tag_weight']
        if 'tfidf_cuml_params' in params:
            self.tfidf_cuml_params = params['tfidf_cuml_params']
            self.tfidf_cuml = CumlTfidfVectorizer(**self.tfidf_cuml_params)
        return self

In [64]:
def debug_vector_dimensions(model, train_df):
    """Выводит отладочную информацию о размерностях векторов в модели.

    Помогает понять структуру и размерность векторов, создаваемых Pipeline, особенно CombinedVectorizer.

    Аргументы:
        model: Обученная модель Pipeline с этапом CombinedVectorizer.
        train_df (pd.DataFrame): DataFrame обучающей выборки.
    """
    if 'vectorizer' not in model.named_steps:
        print("❌ Ошибка: В модели отсутствует этап 'vectorizer'.")
        return

    vectorizer = model.named_steps['vectorizer']

    train_vectors = model.transform(train_df)
    print(f"📐 Форма общих векторов обучающей выборки: {train_vectors.shape}")

    owners_vector_size = 1
    if hasattr(vectorizer, 'mlb') and hasattr(vectorizer.mlb, 'classes_'):
        tags_vector_size = len(vectorizer.mlb.classes_)
    else:
        tags_vector_size = 0
        print("⚠️ Предупреждение: Не удалось определить размерность векторов тегов.")

    print(f"📏 Размерность векторизации владельцев: {owners_vector_size}")
    print(f"📏 Размерность векторизации тегов: {tags_vector_size}")

    start_index_topics = owners_vector_size + tags_vector_size
    end_index_topics = train_vectors.shape[1]
    print(f"📍 Начальный индекс тематических векторов: {start_index_topics}")
    print(f"🏁 Конечный индекс тематических векторов: {end_index_topics}")

    train_topic_vectors = train_vectors[:, start_index_topics:end_index_topics]
    print(f"📐 Форма тематических векторов обучающей выборки: {train_topic_vectors.shape}")

    if hasattr(vectorizer, 'transformed_owners_vectors'):
        print("📏 Размер векторов владельцев (внутренний):", vectorizer.transformed_owners_vectors.shape)
    if hasattr(vectorizer, 'transformed_tags_vectors'):
        print("📏 Размер векторов тегов (внутренний):", vectorizer.transformed_tags_vectors.shape)
    if hasattr(vectorizer, 'transformed_desc_vectors'):
        print("📏 Размер тематических векторов (внутренний):", vectorizer.transformed_desc_vectors.shape)
    if hasattr(vectorizer, 'transformed_combined_vectors'):
        print("📏 Размер объединенных векторов (внутренний):", vectorizer.transformed_combined_vectors.shape)

In [65]:
def get_recommendations_for_game(model, train_df, game_name, top_n=5):
    """Генерирует рекомендации для заданной игры на основе косинусной схожести.

    Использует обученную модель для векторизации игр и расчета косинусной схожести для генерации рекомендаций.

    Аргументы:
        model: Обученная модель Pipeline.
        train_df (pd.DataFrame): DataFrame обучающей выборки.
        game_name (str): Название игры, для которой нужны рекомендации.
        top_n (int, optional): Количество рекомендуемых игр. По умолчанию 5.

    Возвращает:
        list: Список названий рекомендуемых игр. Возвращает пустой список, если игра не найдена или train_df не содержит колонку 'name'.
    """
    if 'name' not in train_df.columns:
        print("❌ Ошибка: DataFrame train_df не содержит колонку 'name'.")
        sys.stdout.flush()
        return []

    game_row = train_df[train_df['name'] == game_name]

    if game_row.empty:
        print(f"⚠️ Игра '{game_name}' не найдена в train_df.")
        sys.stdout.flush()
        return []

    game_vector = model.transform(game_row)
    train_vectors = model.transform(train_df)

    similarity_scores = cosine_similarity(game_vector, train_vectors)[0]

    similarity_df = pd.DataFrame({'name': train_df['name'], 'similarity': similarity_scores})

    sorted_similarity_df = similarity_df.sort_values(by='similarity', ascending=False)

    recommendations_df = sorted_similarity_df[sorted_similarity_df['name'] != game_name].head(top_n)

    recommendations = recommendations_df['name'].tolist()

    return recommendations

In [66]:
def manual_hyperparameter_search(train_df, param_grid, results_file="manual_coherence_results.csv"):
    """Выполняет ручной поиск гиперпараметров на основе когерентности.

    Перебирает заданную сетку параметров, обучает модель для каждой комбинации параметров,
    оценивает когерентность темы и сохраняет результаты.

    Аргументы:
        train_df (pd.DataFrame): DataFrame обучающей выборки.
        param_grid (dict): Сетка параметров для поиска.
        results_file (str, optional): Путь к файлу для сохранения результатов CSV. По умолчанию "manual_coherence_results.csv".

    Возвращает:
        pd.DataFrame: DataFrame с результатами поиска по гиперпараметрам.
    """
    results = []
    grid = ParameterGrid(param_grid)
    total_iterations = len(grid)
    for i, params in enumerate(grid):
        print('------------------------------------------------------------------------------------------')

        BLUE = '\033[94m'
        RESET = '\033[0m'

        start_time = time.time()
        print(f"🧪 Оценка параметров: {params} \n🔄 Итерация: ({i + 1}/{total_iterations})")
        try:
            vectorizer_params = {k.split('__')[1]: v for k, v in params.items() if 'vectorizer__' in k}
            vectorizer = CombinedVectorizer(**vectorizer_params)

            pipeline = Pipeline([('vectorizer', vectorizer)])
            pipeline.fit(train_df)

            tfidf_matrix_cuml = vectorizer.tfidf_cuml.transform(train_df['short_description_clean'])

            row_sums_cp = tfidf_matrix_cuml.sum(axis=1)

            row_sums = row_sums_cp.get()

            zero_vector_indices = np.where(row_sums == 0)[0]

            if len(zero_vector_indices) > 0:
                print(f"⚠️ Обнаружены нулевые векторы TF-IDF для {len(zero_vector_indices)} игр:")
                zero_vector_indices_np = np.array(zero_vector_indices)
                zero_vector_game_ids = train_df.iloc[zero_vector_indices_np].index.tolist()
                print(f"🆔 ID игр с нулевыми векторами: {zero_vector_game_ids}")

            tfidf_model = vectorizer.tfidf_cuml

            model = None
            diversity = -1
            intra_topic_diversity = -1
            model_type = None
            if vectorizer.nmf_params:
                model = vectorizer.nmf
                model_type = 'nmf'
                print("🎭 Темы NMF:")
                if hasattr(model, 'components_'):
                    feature_names = tfidf_model.get_feature_names()
                    if isinstance(feature_names, cudf.core.series.Series):
                        feature_names = feature_names.to_pandas().tolist()
                    display_topics_with_diversity(model, feature_names)
                else:
                    print("⚠️ Модель NMF не имеет атрибута components_")
            elif vectorizer.lda_params:
                model = vectorizer.lda
                model_type = 'lda'
                print("🎭 Темы LDA:")
                if hasattr(model, 'components_'):
                    feature_names = tfidf_model.get_feature_names()
                    if isinstance(feature_names, cudf.core.series.Series):
                        feature_names = feature_names.to_pandas().tolist()
                    display_topics_with_diversity(model, feature_names)
                else:
                    print("⚠️ Модель LDA не имеет атрибута components_")
            else:
                raise ValueError("❌ NMF или LDA не настроены в CombinedVectorizer.")

            texts_for_coherence = train_df['short_description_clean'].tolist()

            coherence = calculate_topic_coherence(model, tfidf_model, texts_for_coherence)

            diversity = -1
            intra_topic_diversity = -1

            if model is not None:
                if hasattr(model, 'components_'):
                    if np.isnan(model.components_).any():
                        print("⚠️ Обнаружены NaN значения в model.components_ перед вычислением diversity!")

            diversity = calculate_topic_diversity(model)

            if hasattr(model, 'components_'):
                feature_names = tfidf_model.get_feature_names()
                if isinstance(feature_names, cudf.core.series.Series):
                    feature_names = feature_names.to_pandas().tolist()
                intra_topic_diversity = calculate_intra_topic_diversity(model, feature_names)

            end_time = time.time()
            recommendations_stellaris = get_recommendations_for_game(pipeline, train_df, "Stellaris", top_n=5)

            results.append({
                'params': params,
                'coherence': coherence,
                'topic_diversity': diversity,
                'intra_topic_diversity': intra_topic_diversity,
                'time': end_time - start_time,
                'recommendations_stellaris': recommendations_stellaris
            })

            output_string = \
            f"""
            {BLUE}Когерентность:{RESET}{BLUE}{coherence:>20.4f}{RESET}
            {BLUE}Разнообразие тем:{RESET}{BLUE}{diversity:>17.4f}{RESET}
            {BLUE}Внутритопиковое разнообразие:{RESET}{BLUE}{intra_topic_diversity:>4.4f}{RESET}
            {BLUE}Время:{RESET}{BLUE}{end_time - start_time:>24.2f} секунд{RESET}
            {BLUE}Рекомендации для Stellaris:{RESET}{BLUE}{', '.join(recommendations_stellaris) if recommendations_stellaris else 'Нет рекомендаций'}{RESET}
            """
            print(output_string)

        except Exception as e:
            end_time = time.time()
            print(f"❌ Ошибка с параметрами {params}: {e}, время: {end_time - start_time:.2f} секунд")
            results.append({
                'params': params,
                'coherence': -1,
                'topic_diversity': -1,
                'intra_topic_diversity': -1,
                'time': end_time - start_time,
                'recommendations_stellaris': []
            })

        results_df = pd.DataFrame(results)
        results_df.to_csv(results_file, index=False)
        print(f"💾 Результаты сохранены в {results_file}")

    return results_df

In [67]:
def train_best_model(train_df, best_params, model_path="best_model.pkl"):
    """Обучает лучшую модель на всем обучающем наборе данных.

    Использует лучшие параметры, найденные в результате поиска, для обучения финальной модели и сохраняет ее.

    Аргументы:
        train_df (pd.DataFrame): Полный обучающий набор данных.
        best_params (dict): Лучшие параметры, найденные в результате поиска гиперпараметров.
        model_path (str, optional): Путь для сохранения обученной модели. По умолчанию "best_model.pkl".

    Возвращает:
        Pipeline: Обученная лучшая модель.
    """
    print("🚀 Начало обучения лучшей модели...")
    start_time = time.time()

    best_model = Pipeline([
        ('vectorizer', CombinedVectorizer()),
    ])
    best_model.set_params(**best_params)
    best_model.fit(train_df)

    with open(model_path, 'wb') as f:
        pickle.dump(best_model, f)
    print(f"💾 Лучшая модель сохранена по пути: {model_path}")

    end_time = time.time()
    print(f"✅ Обучение лучшей модели завершено за {end_time - start_time:.2f} секунд")
    return best_model

In [68]:
param_grid_nmf = {
    'vectorizer__owners_method': ['log_scale'],
    'vectorizer__multilabel_params': [{'sparse_output': True}],
    'vectorizer__nmf_params': [
        {'n_components': 5, 'init': 'nndsvda', 'solver': 'mu', 'beta_loss': 'frobenius'},
        {'n_components': 25, 'init': 'nndsvda', 'solver': 'mu', 'beta_loss': 'frobenius'},
        {'n_components': 50, 'init': 'nndsvda', 'solver': 'mu', 'beta_loss': 'frobenius'},
        {'n_components': 100, 'init': 'nndsvda', 'solver': 'mu', 'beta_loss': 'frobenius'},
        {'n_components': 200, 'init': 'nndsvda', 'solver': 'mu', 'beta_loss': 'frobenius'},
    ],
    'vectorizer__lda_params': [None],
    'vectorizer__tfidf_cuml_params': [{'max_features': 10000}],
    'vectorizer__tag_weight': [1.0]
}

In [32]:
param_grid_lda = {
    'vectorizer__owners_method': ['log_scale'],
    'vectorizer__multilabel_params': [{'sparse_output': True}],
    'vectorizer__nmf_params': [None],
    'vectorizer__lda_params': [
        {'n_components': 10, 'learning_method': 'batch', 'random_state': 42},
        {'n_components': 10, 'learning_method': 'online', 'learning_offset': 10., 'random_state': 42},
        {'n_components': 50, 'learning_method': 'batch', 'random_state': 42},
        {'n_components': 50, 'learning_method': 'online', 'learning_offset': 10., 'random_state': 42},
        {'n_components': 150, 'learning_method': 'batch', 'random_state': 42},
        {'n_components': 150, 'learning_method': 'online', 'learning_offset': 10., 'random_state': 42},
    ],
    'vectorizer__tfidf_cuml_params': [{'max_features': 10000}],
    'vectorizer__tag_weight': [1.0]
}

In [33]:
param_grid_tfidf_max_features = {
    'vectorizer__owners_method': ['log_scale'],
    'vectorizer__multilabel_params': [{'sparse_output': True}],
    'vectorizer__nmf_params': [
        {'n_components': 50, 'init': 'nndsvda', 'solver': 'mu', 'beta_loss': 'frobenius'}
    ],
    'vectorizer__lda_params': [None],
    'vectorizer__tfidf_cuml_params': [
        {'max_features': None},
        {'max_features': 2000},
        {'max_features': 5000},
        {'max_features': 10000},
        {'max_features': 20000}
    ],
    'vectorizer__tag_weight': [1.0]
}

In [34]:
param_grid_tag_weight = {
    'vectorizer__owners_method': ['log_scale'],
    'vectorizer__multilabel_params': [{'sparse_output': True}],
    'vectorizer__nmf_params': [
        {'n_components': 50, 'init': 'nndsvda', 'solver': 'mu', 'beta_loss': 'frobenius'}
    ],
    'vectorizer__lda_params': [None],
    'vectorizer__tfidf_cuml_params': [{'max_features': 10000}],
    'vectorizer__tag_weight': [
        0.5,
        1.0,
        2.0,
        3.0,
        5.0
    ]
}

In [ ]:
manual_results_nmf = manual_hyperparameter_search(
    train_df, param_grid_nmf, results_file="manual_coherence_results_nmf.csv"
)

In [ ]:
print("\n📊 Все результаты поиска NMF:")
print(manual_results_nmf)

In [37]:
best_manual_result_nmf = manual_results_nmf.sort_values(by='coherence', ascending=False).iloc[0]
best_manual_params_nmf = best_manual_result_nmf['params']

In [ ]:
print(f"\n🏆 Лучшие параметры NMF, найденные поиском на основе когерентности: {best_manual_params_nmf}")

In [ ]:
manual_results_lda = manual_hyperparameter_search(
    train_df, param_grid_lda, results_file="manual_coherence_results_lda.csv"
)

In [ ]:
print("\n📊 Все результаты поиска LDA:")
print(manual_results_lda)

In [41]:
best_manual_result_lda = manual_results_lda.sort_values(by='coherence', ascending=False).iloc[0]
best_manual_params_lda = best_manual_result_lda['params']

In [ ]:
print(f"\n🏆 Лучшие параметры LDA, найденные поиском на основе когерентности: {best_manual_params_lda}")

In [ ]:
manual_results_max_features = manual_hyperparameter_search(
    train_df, param_grid_nmf, results_file="manual_coherence_results_max_features.csv"
)

In [ ]:
print("\n📊 Все результаты поиска Max Features:")
print(manual_results_max_features)

In [45]:
best_manual_result_max_features = manual_results_max_features.sort_values(by='coherence', ascending=False).iloc[0]
best_manual_params_max_features = best_manual_result_max_features['params']

In [ ]:
print(f"\n🏆 Лучшие параметры Max Features, найденные поиском на основе когерентности: {best_manual_params_max_features}")

In [ ]:
manual_results_tag_weight = manual_hyperparameter_search(
    train_df, param_grid_tag_weight, results_file="manual_coherence_results_tag_weight.csv"
)

In [ ]:
print("\n📊 Все результаты поиска Tag Weight:")
print(manual_results_max_features)

In [49]:
best_manual_result_tag_weight = manual_results_tag_weight.sort_values(by='coherence', ascending=False).iloc[0]
best_manual_params_tag_weight = best_manual_result_tag_weight['params']

In [ ]:
print(f"\n🏆 Лучшие параметры Tag Weight, найденные поиском на основе когерентности: {best_manual_params_tag_weight}")

---

In [34]:
best_manual_params = {
    'vectorizer__owners_method': ['log_scale'],
    'vectorizer__multilabel_params': [{'sparse_output': True}],
    'vectorizer__nmf_params': 
        [{'n_components': 50, 'init': 'nndsvda', 'solver': 'mu', 'beta_loss': 'frobenius'}],
    'vectorizer__lda_params': [None],
    'vectorizer__tfidf_cuml_params': [{'max_features': 10000}],
    'vectorizer__tag_weight': [2.5]
}

In [ ]:
manual_result = manual_hyperparameter_search(
    train_df, best_manual_params, results_file="manual_coherence_result.csv"
)

In [36]:
manual_result = manual_result.sort_values(by='coherence', ascending=False).iloc[0]
manual_params = manual_result['params']

In [ ]:
manual_model = train_best_model(train_df, manual_params, model_path=model_path)